In [1]:
import sagemaker
import boto3

import numpy as np                                # For performing matrix operations and numerical processing
import pandas as pd                               # For manipulating tabular data
from time import gmtime, strftime
import os

region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')

In [19]:
from sagemaker import get_execution_role

role = 'arn:aws:iam::597261055787:role/service-role/AmazonSageMaker-ExecutionRole-20210708T115429'
print(role)

arn:aws:iam::597261055787:role/service-role/AmazonSageMaker-ExecutionRole-20210708T115429


In [20]:
sess = sagemaker.Session()
bucket = 'hyperparam'                   # Set a default S3 bucket
prefix = '1stTrial'

In [21]:
print(bucket)

hyperparam


In [22]:
# Import raw data 
def import_raw_data():
    raw = pd.read_csv('csv//true_car_listings.csv')
    df = raw[['Price','Mileage','Year','Make']].copy().dropna()
    df['Make'] = df['Make'].rank() 
    return df 
df = import_raw_data()
df.to_csv('csv//dataset.csv')

In [23]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'dataset.csv')).upload_file('csv//dataset.csv')

S3UploadFailedError: Failed to upload csv//dataset.csv to hyperparam/1stTrial/dataset.csv: An error occurred (AccessDenied) when calling the CreateMultipartUpload operation: Access Denied